<a href="https://colab.research.google.com/github/quang-m-nguyen/DeepPGD/blob/main/dna_bert2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

!pip install einops

from transformers.models.bert.configuration_bert import BertConfig
import torch
from transformers import AutoTokenizer, AutoModel

config = BertConfig.from_pretrained("zhihan1996/DNABERT-2-117M")
model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True, config=config)
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

bert_layers.py:   0%|          | 0.00/40.7k [00:00<?, ?B/s]

flash_attn_triton.py:   0%|          | 0.00/42.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNABERT-2-117M:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

pytorch_model.bin:   0%|          | 0.00/468M [00:00<?, ?B/s]

/root/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/d064dece8a8b41d9fb8729fbe3435278786931f1/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at zhihan1996/DNABERT-2-117M and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/168k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
def get_embedding(dna):
  inputs = tokenizer(dna, return_tensors = 'pt')["input_ids"]
  hidden_states = model(inputs)[0] # [1, sequence_length, 768]
  embedding_mean = torch.mean(hidden_states[0], dim=0)
  return embedding_mean

dna = "ACGTAGCATCGGATCTATCTATCGACACTTGGTTATCGATCTACGAGCATCTCGTTAGC"
embedding = get_embedding(dna)
print(len(embedding))

768


In [4]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/deepPGD/4mC/4mC_C.equisetifolia/train.tsv', delimiter='\t')
df_limited = df.head(5)
display(df_limited)

,index,label,text
0,0,1,ATTAACAGTTTCCATAAAATCGGGACTAGCTGTCCAAAAAT
1,1,1,GAGAATAAGACTTATTCTCTCAGCAAGTAGTTTTGAGATCA
2,2,1,ATGTAGAGACAAACTACCTACAGCAGTAGTGGTTTCTCGTT
3,3,1,ATTTTTACCATCATAACATTCGTTATCAGTGGTCTTGTGGA
4,4,1,ATACAAGTATCACTTATATACAGAATTTCAAAGAGACCGTT


In [5]:
import pickle
import os
import pandas as pd

def process_tsv_and_save_embeddings(tsv_file_path):
    # Read the TSV file
    df = pd.read_csv(tsv_file_path, delimiter='\t')
    embeddings_data = []
    count = 0

    # Process each DNA sequence
    for index, row in df.iterrows():
        dna = row['text']
        embedding = get_embedding(dna)
        embeddings_data.append({'dna': dna, 'embedding': embedding})
        count += 1
        print(f"Finished processing DNA sequence {count}: {dna}, embedding shape: {embedding.shape}")

    # Determine the output file path
    tsv_dir = os.path.dirname(tsv_file_path)
    tsv_filename = os.path.basename(tsv_file_path).replace('.tsv', '')
    output_file_path = os.path.join(tsv_dir, f"{tsv_filename}_embeddings_data.pkl")

    # Save the embeddings data as a pickle file
    with open(output_file_path, 'wb') as f:
        pickle.dump(embeddings_data, f)

    print(f"Embeddings data saved to {output_file_path}")

# Example usage
# tsv_file_path = '/content/drive/MyDrive/deepPGD/4mC/4mC_C.equisetifolia/test.tsv'
# process_tsv_and_save_embeddings(tsv_file_path)

In [6]:
# List of TSV file paths to process
tsv_file_paths = [
    '/content/drive/MyDrive/deepPGD/4mC/4mC_F.vesca/test.tsv',
    '/content/drive/MyDrive/deepPGD/4mC/4mC_F.vesca/train.tsv'



    # '/content/drive/MyDrive/deepPGD/6mA/6mA_Xoc BLS256/test.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_D.melanogaster/test.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_A.thaliana/test.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_H.sapiens/test.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_C.elegans/test.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_T.thermophile/test.tsv',
    # '/content/drive/MyDrive/deepPGD/5hmC/5hmC_H.sapiens/test.tsv',
    # '/content/drive/MyDrive/deepPGD/4mC/4mC_S.cerevisiae/test.tsv',
    # '/content/drive/MyDrive/deepPGD/4mC/4mC_F.vesca/test.tsv',
    # '/content/drive/MyDrive/deepPGD/4mC/4mC_C.equisetifolia/test.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_Xoc BLS256/train.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_D.melanogaster/train.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_A.thaliana/train.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_H.sapiens/train.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_C.elegans/train.tsv',
    # '/content/drive/MyDrive/deepPGD/6mA/6mA_T.thermophile/train.tsv',
    # '/content/drive/MyDrive/deepPGD/5hmC/5hmC_H.sapiens/train.tsv',
    # '/content/drive/MyDrive/deepPGD/4mC/4mC_S.cerevisiae/train.tsv',
    # '/content/drive/MyDrive/deepPGD/4mC/4mC_F.vesca/train.tsv',
    # '/content/drive/MyDrive/deepPGD/4mC/4mC_C.equisetifolia/train.tsv'
]

# Loop through each file path and process it
for tsv_file_path in tsv_file_paths:
    process_tsv_and_save_embeddings(tsv_file_path)

Streaming output truncated to the last 5000 lines.
Finished processing DNA sequence 10800: TTAGCAAACCAATAGATATACCATTAAGTATGAACAACGCT, embedding shape: torch.Size([768])
Finished processing DNA sequence 10801: GCCGGTGCCGGAGATTGAAACAGAGGCAGCAGAACCGCCGC, embedding shape: torch.Size([768])
Finished processing DNA sequence 10802: CGCACGGTTTCAGTGTACTGCTTCTCTATGTGGCAGTACAC, embedding shape: torch.Size([768])
Finished processing DNA sequence 10803: ACTTATTGCTTCCATAATATCAGATTTATTGAAGGGACTAA, embedding shape: torch.Size([768])
Finished processing DNA sequence 10804: TAAGTATCTTAACAACATATCATGTGATCATGTAATTTCAC, embedding shape: torch.Size([768])
Finished processing DNA sequence 10805: TTCTTGAGTTGGTTGCACTACACATCCCTCATGGTATGCAA, embedding shape: torch.Size([768])
Finished processing DNA sequence 10806: GGAGAGAATTTCCCCCTTACCTCTCTCAATCTATAGTGATA, embedding shape: torch.Size([768])
Finished processing DNA sequence 10807: AGCAAATCAACGAGTAATGACATCCACCCCACACTTAAAAT, embedding shape: torch.Size([768])
Finis

In [ ]:
import pickle

def read_embeddings_data(pkl_file_path, num_rows=1):
    with open(pkl_file_path, 'rb') as f:
        embeddings_data = pickle.load(f)

    limited_data = embeddings_data[:num_rows]
    for item in limited_data:
        print(f"DNA: {item['dna']}, Embedding shape: {item['embedding'].shape}")

# Example usage
pkl_file_path = '/content/drive/MyDrive/deepPGD/4mC/4mC_F.vesca/train_embeddings_data.pkl'
read_embeddings_data(pkl_file_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/deepPGD/4mC/4mC_F.vesca/train_embeddings_data.pkl'